In [ ]:
import pandas as pd
import numpy as np
import random

# Load data

In [ ]:
html_file_name = 'data/2021-06-02.html'
data = pd.read_html(html_file_name)

# Review data

In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
type(data[0])

In [ ]:
next_matches_index = 0
standings_indexes = range(1,7)
gatekeepers_index = 7
defenders_index = 8
midfielders_index = 9
forwards_index = 10

In [ ]:
data[next_matches_index]

In [ ]:
data[random.randint(min(standings_indexes), max(standings_indexes))]

In [ ]:
data[random.randint(gatekeepers_index, forwards_index)]

# Data reorginaize and pre-process

In [ ]:
next_matches_columns_renames = {"תאריך": "date", "שעה": "hour", "קבוצה מארחת": "home_team", "תוצאה": "score", "קבוצה אורחת": "away_team", "סקירה": "review"}
next_matches_df = data[next_matches_index].rename(columns=next_matches_columns_renames).copy()
next_matches_df